In [1]:
from pymeasure.instruments.newport import ESP300
from pymeasure.instruments import Instrument
# from pymeasure.instruments.validators import strict_discrete_set
from pymeasure.instruments import list_resources
from pymeasure.instruments.lakeshore import LakeShore425
from pymeasure.adapters import VISAAdapter
# import visa

from pymeasure.display import widgets as pw
from pymeasure.display.inputs import *

# from pyvisa.constants import StopBits, Parity

import ftd2xx  # Thorlabs MFF101
import ftd2xx.defines as constants
from ftd2xx import listDevices, getDeviceInfoDetail
import time

import pymeasure.experiment.parameters as pars

In [ ]:
params = pars.FloatParameter(ps,units='mm')

In [ ]:
pars.ListParameter(ps)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
ctrl.FloatInput.set_parameter()

In [ ]:
pymeasure.display.inputs

In [ ]:
ps = x.position

In [2]:
resources = list_resources()

C:\Users\labuser\Anaconda3\envs\controller\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


0 : USB0::0x1313::0x8072::P2010852::INSTR : Thorlabs,PM100USB,P2010852,1.6.0
1 : ASRL1::INSTR : Not known
2 : ASRL3::INSTR : Not known
3 : ASRL6::INSTR :  
4 : GPIB0::3::INSTR : ESP300 Version 3.08 09/09/02


In [3]:
FTDIresources = listDevices()
FTDIresources

[b'37000805', b'18031043']

In [ ]:
getDeviceInfoDetail(1)

## Laser on/off

In [4]:
serial = FTDIresources[0]

In [5]:
def laser(switch):
    """Switch 'on' or 'off'"""
    serial = FTDIresources[0]
    # Raw byte commands for "MGMSG_MOT_MOVE_JOG".
#     on = b"\x6A\x04\x00\x01\x21\x01"  # x01 up
#     off = b"\x6A\x04\x00\x02\x21\x01"  # x02 down
    
    if switch == 'on':
        motor = ftd2xx.openEx(serial)
        print(motor.getDeviceInfo())
        motor.setBaudRate(115200)
        motor.setDataCharacteristics(constants.BITS_8, constants.STOP_BITS_1, constants.PARITY_NONE)
        time.sleep(.05)
        motor.purge()
        time.sleep(.05)
        motor.resetDevice()
        motor.setFlowControl(constants.FLOW_RTS_CTS, 0, 0)
        motor.setRts()

        # Send raw bytes to USB driver.
        motor.write(b"\x6A\x04\x00\x01\x21\x01")  # up or 
        motor.close()
    else:
        motor = ftd2xx.openEx(serial)
        print(motor.getDeviceInfo())
        motor.setBaudRate(115200)
        motor.setDataCharacteristics(constants.BITS_8, constants.STOP_BITS_1, constants.PARITY_NONE)
        time.sleep(.05)
        motor.purge()
        time.sleep(.05)
        motor.resetDevice()
        motor.setFlowControl(constants.FLOW_RTS_CTS, 0, 0)
        motor.setRts()

        # Send raw bytes to USB driver.
        motor.write(b"\x6A\x04\x00\x02\x21\x01")  # up or 
        motor.close()

In [6]:
laser('off')

{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}


## XYZ Control

In [7]:
ctrl = ESP300("GPIB0::3::INSTR")

In [8]:
ctrl.data_bits=8
ctrl.baud_rate=19200
ctrl.StopBits=1
ctrl.read_termination='\r\n'
ctrl.write_termination='\r'

In [9]:
x = ctrl.y  # Our x-axis is motor 2 or y
y = ctrl.x  # Our y-axis is motor 1 or x
z = ctrl.phi  # 3

In [10]:
pos = [x.position, y.position, z.position]
pos

[7.99998, 4.00609, 4.58983]

In [ ]:
x.wait(500)

In [ ]:
x.wait_for_stop()

In [ ]:
laser('off')

In [ ]:
x.write('WS')

In [12]:
vel = Input('Vel?')

AttributeError: 'str' object has no attribute 'default'

In [11]:
x.velocity = 0.1
laser('on')
x.wait(500)
x.position = 7
# while x.motion_done == False:
#     time.sleep(.05)
# x.position = 9
x.wait_for_stop()
laser('off')

{'type': 5, 'id': 67369712, 'description': b'APT Filter Flipper', 'serial': b'37000805'}


VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [ ]:
def laser_off(axis):
    if axis.motion_done:
        laser('off')    
        

In [ ]:
laser_off(x)

In [ ]:
x.position=7
x.motion_done

In [ ]:
x.motion_done

In [ ]:
ctrl.write('1HN2,1')  # y, x grouped for simultaneous movement
ctrl.ask('TB') 

In [ ]:
ctrl.write('2HN1,2')  # TODO: can't assign group #2, out of range ?
ctrl.ask('TB')

In [ ]:
#to create new group
ctrl.write('1HX')
ctrl.write('2HN2,3')  # y, z

In [ ]:
ctrl.write('1HX')  # delete group #1

In [ ]:
ctrl.ask('HB')

In [ ]:
ctrl.ask('1HN?')

In [ ]:
ctrl.ask('1VU?')  # ask whats max group vel for instrument

In [ ]:
ctrl.ask('1HV?')  # current group vel

In [ ]:
ctrl.write('1HV0.2')

In [ ]:
ctrl.ask('1HP?')  # current group pos

In [ ]:
ctrl.write('2HO')

PROBLEM: somehow moving both xy axes simultaneously exceeds threshold.

In [ ]:
ctrl.write('1HL8,4')
ctrl.ask('TB') 

In [ ]:
x.enabled

In [ ]:
x.velocity  # get current scanning speed

In [ ]:
x.velocity = 0.4  # set scanning speed

In [ ]:
x.position  # get position

Problem: when xy axes assigned to group #1, single axes movement not allowed.
Solution: after XY motion done, delete group.


In [ ]:
x.position = 8 # move to position  
ctrl.ask('TB') 